In [1]:
def OCRImgLines(imagePath,folder):
    print(imagePath)
    image = cv2.imread(imagePath)
    text = str(pytesseract.image_to_string(image))
    #print(text)

    text_file = open(folder+'\\'+file[:-4]+'.txt', "w")
    n = text_file.write(text)
    text_file.close()
    text_file = open(folder+'\\'+file[:-4]+'.txt', 'r')
    lines = text_file.readlines()
    text_file.close()
    return lines

def checkVar(toCheck):
    if toCheck is None:
        return ''
    else:
        return toCheck

In [2]:
def dataExtract(toExtract,folder,file):
    imagePath = (folder+'\\'+file)
    nextPageImagePath = (folder+'\\1-'+file[2:])

    #extracts the lines of text from an image
    lines = [line for line in OCRImgLines(imagePath,folder) if line!='\n']
    lines=lines[:-1]#removes pagenumber
    print(lines)
    data = None

    #sets the keyphrase depending on the data found in the page
    if toExtract == 'scope1':
        keyphrase = "Gross global Scope 1 emissions (metric tons CO2e)"
    elif toExtract == 'scope2':
        keyphrase = "Scope 2, location-based"
    elif toExtract == 'scope3':
            keyphrase = "Metric tonnes CO2e"

    '''
    Goes through each line in the page
    if the keyphrase is in the line, the next line will contain the data we want
    saves the next line as the data

    TODO Fix bug with label on last line, and value on first line of next page ^Microsoft
    '''
    
    for i in range(len(lines)):
        #if keyphrase is in ;ine, the next line contains the data
        print('len(lines): '+str(len(lines)))
        print('i+1: '+str(i))
        try:
            if keyphrase in lines[i] and (len(lines)>=i+1):
                data=lines[i+1]
            #if keyphrase is in last line, the first line in the next page contains the data
            elif keyphrase in lines[i] and (len(lines)==i+1):
                print('Keyphrase found in last line: taking first line of next page')
                nextPageLines = [line for line in OCRImgLines(nextPageImagePath,folder) if line!='\n']
                for j in range(len(nextPageLines)):
                    if 'DISCLOSURE INSIGHT ACTION' in nextPageLines[j]:
                        data=nextPageLines[j+1]
        except IndexError:
            data=''

    '''
    If the keyphrase is not in the first page, it checks the second page
    does this until it finds data. 
    '''
    if data == None:
        print('data not found in page, checking next page')
        nextPageLines = [line for line in OCRImgLines(nextPageImagePath,folder) if line!='\n']
        #print(nextPageLines)
        for i in range(len(nextPageLines)):
            if keyphrase in nextPageLines[i] and data ==None:
                data = nextPageLines[i+1]
                    

    return data

In [3]:
from pathlib import Path
def OCRPages(folderpath,pages,numPages):
    Path(folderpath+'\\bin').mkdir(parents=True, exist_ok=True)
    for i in reversed(range(0, numPages)):
        print('---checking page '+str(i))
        pages[i].save(folderpath+'\\bin\\'+str(i)+'.png')
        lines = [line for line in OCRImgLines((folderpath+'\\bin\\'+str(i)+'.png'),folderpath+'\\bin\\') if line!='\n']
        for phrase in filterPhrases:
            for line in lines:
                if phrase in line:
                    pages[i].save(folderpath+'\\0-'+phrase+'.png')
                    pages[i+1].save(folderpath+'\\1-'+phrase+'.png')
    for f in os.listdir(folderpath+'\\bin'):
        os.remove(folderpath+'\\bin\\'+f)
    os.rmdir(folderpath+'\\bin')

In [4]:
import cv2
import os
import re
import pytesseract
from pdf2image import convert_from_path
from os import walk
import csv
import PyPDF2
import warnings
warnings.filterwarnings("error")


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
filteredPages = []
filterPhrases = ['C6.1',
                'C6.3',
                'C6.5']
count=0
dates = ['2019','2020','2021','2022']

folder = r'C:\Users\Clamfighter\Documents\GitHub\Data-Mining\Data-Mining\CDP\pdfs(unf)'

all_files = os.listdir(folder)    
pdf_files = list(filter(lambda f: f.endswith('.pdf'), all_files))

for file in pdf_files:
    companyName=''
    date=''
    try:
        print(file)
        pdfname = file
        folderpath = folder+"\\"+file[:-4]
        print(folderpath)
        if not os.path.exists(folderpath):
            os.makedirs(folderpath)

        #convert the pages to images
        print('converting Pages')
        pages = convert_from_path(folder+"\\"+file, 350)
        print('pages Converted')
        

        '''
        Searching of PDF for wanted data
        '''
        print('Checking Pages for wanted data')
        pdf = PyPDF2.PdfFileReader(folder+"\\"+file,strict=False)

        '''
        Checking first page of data
        '''
        #print(pages)
        pages[0].save(folderpath+'\Page0.png')
        lines = [line for line in OCRImgLines((folderpath+'\Page0.png'),folderpath) if line!='\n']
        #print(lines)
        openingLines = lines[0:3]
        #print(openingLines)
        if any('Climate Change' in line for line in openingLines):
            print('Correct PDF Format')
            sep = 'Climate Change'
            for openingLine in reversed(openingLines):
                for d in dates:
                    if d in openingLine:
                        date=d
                print(openingLine)
                if sep in openingLine:
                    print('Opening Line Contains Name')
                    companyName = openingLine.split(sep, 1)[0]
                    print(companyName)
                    companyName = re.sub('CDP', '', companyName)
                    print(companyName)
        else:
            print('Incorrect format found due to lack of "Climate Change" in first lines of first page')
            continue
        print(companyName)
        print(date)
        
        foundDate=False #used to indicate whether any data was found in any of the pages
        textExtracted=False
        try:
            numPages = pdf.getNumPages()
            for i in reversed(range(0, numPages)):
                print('---checking page '+str(i))
                pageObj = pdf.getPage(i)
                text = pageObj.extractText()
                #print(text)
                for phrase in filterPhrases:
                    #print('---searching for phrase '+phrase)
                    #resSearch = re.search(phrase, text)
                    
                    if phrase in text:
                        #print('Data Found: ', phrase)
                        
                        pages[i].save(folderpath+'\\0-'+phrase+'.png')
                        pages[i+1].save(folderpath+'\\1-'+phrase+'.png')
                        foundData=True
                
                if text!='':
                    textExtracted=True
                        
        except NotImplementedError                      :
            print('PDF Encrypted')
        except Exception as e:
            print(e)
            numPages=len(pages)
            OCRPages(folderpath,pages,numPages)
            textExtracted=True


        if not textExtracted:
            print('Uable to extract text from PDF file. Conducting OCR. This may take some minutes')
            OCRPages(folderpath,pages,numPages)

        print('Data pages Extracted')
        print('Extracting data from pages')
        scope1=''
        scope2=''
        scope3=''
        for file in os.listdir(folderpath):
            if file == '0-C6.1.png':
                scope1 = checkVar(dataExtract('scope1',folderpath,file))
            elif file == '0-C6.3.png':
                scope2 = checkVar(dataExtract('scope2',folderpath,file))
            elif file == '0-C6.5.png':
                scope3 = checkVar(dataExtract('scope3',folderpath,file))
                
        print(scope1)
        print(scope2)
        print(scope3)
        print('Exporting Data')
        row=[date,scope1,scope2,scope3]
        row = [item.strip() for item in row]
        row = [item.replace(' ', '') for item in row]
        row.append(companyName)
        row.append(pdfname)
        f = open(folder+'/output.csv', 'a', newline='')
        writer = csv.writer(f)
        writer.writerow(row)
        f.close()
    except (FileNotFoundError, NotImplementedError,IndexError) as e:
        print('error: '+e)
        row=[date,'','','',companyName,pdfname]
        f = open(folder+'/output.csv', 'a', newline='')
        writer = csv.writer(f)
        writer.writerow(row)
        f.close()

1569909968-CDPClimateChangeQuestionnaire2019.pdf
C:\Users\Clamfighter\Documents\GitHub\Data-Mining\Data-Mining\CDP\pdfs(unf)\1569909968-CDPClimateChangeQuestionnaire2019
converting Pages
pages Converted
Checking Pages for wanted data
C:\Users\Clamfighter\Documents\GitHub\Data-Mining\Data-Mining\CDP\pdfs(unf)\1569909968-CDPClimateChangeQuestionnaire2019\Page0.png
Correct PDF Format
30, 2019

AngloGold Ashanti CDP Climate Change Questionnaire 2019 Monday, September >» > b D Pp

Opening Line Contains Name
AngloGold Ashanti CDP 
AngloGold Ashanti  
qN CDP

AngloGold Ashanti  
2019
---checking page 54
---checking page 53
---checking page 52
---checking page 51
---checking page 50
---checking page 49
---checking page 48
---checking page 47
---checking page 46
---checking page 45
---checking page 44
---checking page 43
---checking page 42
---checking page 41
---checking page 40
---checking page 39
---checking page 38
---checking page 37
---checking page 36
---checking page 35
---checking page

TypeError: Unsupported image object